# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

cpu is available.


In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_cf_'
}

In [34]:
params = param1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [35]:
# get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
from Clust.clust.ML.tool import meta
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

In [36]:
model_meta

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},


In [37]:
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, model_meta, db_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_testX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
window_size: 144 nan_limit_num: 72
(6048, 24) (42, 144, 24)
(42, 1) (42, 1)


In [38]:
# set scaler, feature list by data y flag
scaler_feature_dict = {}
if model_meta['data_y_flag']:
    scaler_feature_dict['scaler'] = scaler_y
    scaler_feature_dict['feature_list'] = model_meta['ingestion_param_y']['feature_list']
    scaler_feature_dict['target'] = model_meta['ingestion_param_y']['feature_list'][0]

else:
    scaler_feature_dict['scaler'] = scaler_X
    scaler_feature_dict['feature_list'] = model_meta['ingestion_param_X']['feature_list']
    scaler_feature_dict['target'] = model_meta['ingestion_param_X']['feature_list'][0]

In [39]:
# model test
result = ML_api.ML_test(model_meta, test_X_array, test_y_array, scaler_feature_dict)

features shape: torch.Size([42, 144, 24]) targets shape:  torch.Size([42, 1])

Start testing data

scale
=========================data
        value  prediction
0   17.370001    7.650870
1   20.650000    7.562680
2   11.420000    7.149018
3   10.680000    7.541431
4   12.439999    7.565460
5   11.170000    7.380070
6   24.120001    7.650904
7   10.990000    7.690277
8   13.760000    7.368565
9   14.560000    7.388246
10  14.970000    7.460073
11  13.429999    7.464605
12  11.570000    7.626628
13   9.330000    7.931899
14  15.580000    7.759978
15  20.929998    7.646781
16  11.990000    7.403240
17  15.370000    7.682791
18  10.470000    7.525136
19  14.410000    7.600495
20  10.160000    7.498172
21  15.120000    7.460516
22  12.320000    7.555345
23  10.460000    7.281071
24  15.060000    6.951315
25   9.680000    7.198527
26  10.690000    7.611364
27  17.799997    7.626010
28  10.690000    7.720323
29  17.059998    7.561688
30  20.879999    7.566436
31  10.890000    7.503093


In [40]:
result

{'result': {'index': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31],
  'value': {'value': [17.3700008392334,
    20.649999618530273,
    11.420000076293945,
    10.680000305175781,
    12.439998626708984,
    11.170000076293945,
    24.1200008392334,
    10.989999771118164,
    13.760000228881836,
    14.5600004196167,
    14.970000267028809,
    13.429999351501465,
    11.569999694824219,
    9.329999923706055,
    15.579999923706055,
    20.92999839782715,
    11.989999771118164,
    15.369999885559082,
    10.470000267028809,
    14.40999984741211,
    10.15999984741211,
    15.119999885559082,
    12.319999694824219,
    10.460000038146973,
    15.0600004196167,
    9.680000305175781,
    10.6899995803833,
    17.799997329711914,
    10.6899995803833,
    17.05999755859375,
    20.8799991607666,
    10.890000343322754],

In [26]:
model_meta['model_info']

{'model_purpose': 'regression',
 'model_method': 'LSTM_rg',
 'model_name': 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'rnn_type': 'lstm',
  'input_size': 3,
  'hidden_size': 64,
  'num_layers': 2,
  'output_dim': 1,
  'dropout': 0.1,
  'bidirectional': True},
 'seq_len': 24,
 'input_size': 3,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/LSTM_rg/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_rg_/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX/model.pkl'}}}